In [1]:
#! /u1/vian294/julia/julia-1.1.0/bin/julia -p 8 
#PBS -N jHexaG1e3p08
#PBS -l select=1:ncpus=8
#PBS -j oe
#PBS -k oe

# qsub /u1/vian294/METMAT/AutoDiff/rev_paper1/example_3DSpring_r1.jl

In [2]:
using Printf, WriteVTK, AbaqusReader, JLD, Dates, LinearAlgebra

In [12]:
using Logging

Logging.disable_logging(LogLevel(0))

In [3]:
let sPath = "/home/andrea/Documents/github/AD4SM.jl/src/"
# let sPath = "C:\\Users\\vian294\\OneDrive - CIRA\\Documents\\github\\AD4SM.jl\\src\\"
  include(sPath*"adiff.jl")
  include(sPath*"materials.jl")
  include(sPath*"elements.jl")
end

Main.Elements

In [4]:
# @show Elements.setp(8)

mean(x)   = sum(x)/length(x)

# sMeshFile = "phole3D_h0500.inp" 
sMeshFile = "phole3D_h0330.inp"
# mat       = Materials.NeoHooke(10, 1e3)
mat       = Materials.Ogden(8,1,1e3)

bisinc    = true
sVTKpath  = "./vtk_files/"
bVTKall   = true
sPosFix   = "_1e3w250"
# ϵn0       = -0.24

N         = 30 
LF        = range(0, 1, length=N)
#LF        = vcat(range(0.0, 0.7, length=N÷3),
#                 range(0.7, 1.0, length=2N÷3+1))
nSteps    = length(LF)
;

In [13]:
sFileName = splitext(sMeshFile)[1]*sPosFix
mymodel   = AbaqusReader.abaqus_read_mesh(sMeshFile)
nodes     = [mymodel["nodes"][ii] for ii in 1:mymodel["nodes"].count]
el_nodes  = [item[2]              for item in mymodel["elements"]] 



id_l = mymodel["node_sets"]["left_bnd"]
id_r = mymodel["node_sets"]["right_bnd"]
id_t = mymodel["node_sets"]["top_bnd"]
id_b = mymodel["node_sets"]["bottom_bnd"]


n_t, n_b  = length(id_t), length(id_b)
;

┌ Warning: Unknown section: 'HEADING'
└ @ AbaqusReader /home/andrea/.julia/packages/AbaqusReader/C1mY0/src/parse_mesh.jl:270


In [14]:
(xmin, xmax) = extrema([item[2][1] for item ∈ mymodel["nodes"]])
(ymin, ymax) = extrema([item[2][2] for item ∈ mymodel["nodes"]])
(zmin, zmax) = extrema([item[2][3] for item ∈ mymodel["nodes"]])
@show Δx, Δy, Δz      = (xmax-xmin), (ymax-ymin), (zmax-zmin)
;

(Δx, Δy, Δz) = (xmax - xmin, ymax - ymin, zmax - zmin) = (5.0, 5.0, 1.0)


In [15]:
@show nNodes, nElems  = length(nodes), length(el_nodes)
;

(nNodes, nElems) = (length(nodes), length(el_nodes)) = (1490, 984)


In [16]:
mat       = Materials.Ogden(16,1,1e2)

Main.Materials.Ogden(16.0, 1.0, 100.0)

In [17]:
points    = hcat(nodes...)
cells     = [if length(item)==4
               MeshCell(VTKCellTypes.VTK_TETRA, item)
             else 
               MeshCell(VTKCellTypes.VTK_HEXAHEDRON, item)
             end for item in el_nodes]
elems     = [if length(item)==4
               Elements.Tet04(item, nodes[item], mat=mat)
             else 
               Elements.Hex08(item, nodes[item], mat=mat)
             end for item  in el_nodes]
;

In [18]:
@printf("starting %s \n\n", sFileName) 

ifree = trues(3, nNodes)  
unew  = zeros(3, nNodes)
λnew  = zeros(6)
idxes = LinearIndices(unew)
fnew  = zeros(3, nNodes)
allus = []
t0    = Base.time_ns()
;

starting phole3D_h0330_1e3w250 



In [23]:
ϵn0       = 0.5
;

In [24]:
u              = fill(NaN, 3, nNodes)
u[:,id_b]     .= 0
u[1,id_t]     .= 0 
u[3,id_t]     .= 0
u[2,id_t]     .= ϵn0*Δy

ifree     = isnan.(u)
icnst     = .!ifree
;

In [25]:
u0         = 1e-6Δy*randn(3, nNodes)
u0[icnst] .= 0
unew       = copy(u0)
@time Elements.solvestep!(elems, u0, unew, ifree, bprogress=false, becho=true)
u[ifree] .= unew[ifree]
;


predictor step done in 2.66 sec., with normupdt: 1.85e-05, starting corrector loop
iter:  0, norm0: 1.34e-05, normru: 8.00e-01, normre: NaN, normupdt: 3.70e-05, α: -1.000, eltime: 3.01 sec.
iter:  1, norm0: 1.20e-09, normru: 6.87e-10, normre: NaN, eltime: 2.48 sec.
  8.153007 seconds (4.03 M allocations: 29.440 GiB, 11.42% gc time)


### Do the compressive branch

In [26]:
N       = 20
#LF_c    = vcat(range(0.0, 0.9, length=2N÷8),
#               range(0.9, 1.0, length=6N÷8))
LF_c    = range(0.0, 1.0, length=N)

allus_c = Elements.solve(elems, u, LF=LF_c, ifree=ifree, 
  bprogress=false, becho=true, bechoi=false)
;

step   1/20, LF=0.000, done in  1 iter, after 5.46 sec.
step   2/20, LF=0.053, done in  4 iter, after 13.16 sec.
step   3/20, LF=0.105, done in  3 iter, after 10.60 sec.
step   4/20, LF=0.158, done in  3 iter, after 11.14 sec.
step   5/20, LF=0.211, done in  3 iter, after 10.69 sec.
step   6/20, LF=0.263, done in  3 iter, after 10.57 sec.
step   7/20, LF=0.316, done in  3 iter, after 10.69 sec.
step   8/20, LF=0.368, done in  2 iter, after 7.66 sec.
step   9/20, LF=0.421, done in  2 iter, after 8.32 sec.
step  10/20, LF=0.474, done in  2 iter, after 8.48 sec.
step  11/20, LF=0.526, done in  2 iter, after 7.96 sec.
step  12/20, LF=0.579, done in  2 iter, after 8.89 sec.
step  13/20, LF=0.632, done in  2 iter, after 8.41 sec.
step  14/20, LF=0.684, done in  2 iter, after 8.21 sec.
step  15/20, LF=0.737, done in  2 iter, after 7.84 sec.
step  16/20, LF=0.789, done in  2 iter, after 7.50 sec.
step  17/20, LF=0.842, done in  2 iter, after 7.70 sec.
step  18/20, LF=0.895, done in  2 iter, af

In [28]:
paraview_collection(sVTKpath*sFileName) do pvd
  for (ii, item) in enumerate(allus_c)
    (u0,rf) = item[1:2]
    F   = Elements.getinfo(elems,u0,info=:F)
    LE  = Elements.getinfo(elems,u0,info=:LE)
    J   = [det(F) for F in F]
    E   = [0.5*(transpose(F)*F-I) for F in F]
    σ   = [Materials.getinfo(F,mat,info=:σ)        for F in F]  # Cauchy stress
    S   = [Materials.getinfo(F,mat,info=:S)        for F in F]  # 2nd PK
    Ii  = [Materials.getInvariants(transpose(F)F)  for F in F]  # 2nd PK
    σP  = [sort(eigvals(σ))  for σ  in σ]
    σVM = [sqrt((x[1]-x[2])^2+(x[2]-x[3])^2+(x[3]-x[1])^2)/sqrt(2)  for x in σP]

    vtkobj  = vtk_grid(@sprintf("%s_%03i", sVTKpath*sFileName, ii), points+u0, cells)
    vtk_point_data(vtkobj, (u0[1,:],u0[2,:],u0[3,:]),                   "u")
    vtk_point_data(vtkobj, (rf[1,:],rf[2,:],rf[3,:]),                   "rf")
    vtk_cell_data(vtkobj,  tuple([getindex.(F,ii)  for ii in 1:9]...),  "F")
    vtk_cell_data(vtkobj,  tuple([getindex.(LE,ii)  for ii in 1:9]...), "LE")
    vtk_cell_data(vtkobj,  J,   "J")
    vtk_cell_data(vtkobj,  tuple([getindex.(σ,ii)   for ii in 1:9]...), "\$\\sigma\$")
    vtk_cell_data(vtkobj,  tuple([getindex.(σP,ii)  for ii in 1:3]...), "\$\\sigma_p\$")
    vtk_cell_data(vtkobj,  tuple([getindex.(Ii,ii)  for ii in 1:3]...), "I")
    vtk_cell_data(vtkobj,  σVM, "\$\\sigma_{VM}\$")

    collection_add_timestep(pvd, vtkobj, ii)
    vtk_save(vtkobj)
  end
end  

21-element Array{String,1}:
 "./vtk_files/phole3D_h0330_1e3w250.pvd"
 "./vtk_files/phole3D_h0330_1e3w250_001.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_002.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_003.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_004.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_005.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_006.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_007.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_008.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_009.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_010.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_011.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_012.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_013.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_014.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_015.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_016.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_017.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_018.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_019.vtu"
 "./vtk_files/phole3D_h0330_1e3w250_020.vtu"